In [1]:
import pandas as pd
from datasets import load_dataset
from transformers import pipeline
from googletrans import Translator
import random

/home/wolf/anaconda3/envs/spero-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-18 14:04:09.836699: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-18 14:04:10.384759: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-18 14:04:13.371828: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slight

AttributeError: _ARRAY_API not found

In [2]:
dataset = load_dataset('lvimuth/HealthRisk-1500-Medical-Risk-Prediction')

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Patient_ID', 'Age', 'Gender', 'Symptoms', 'Medical_History', 'Medications', 'Lab_Reports', 'Lifestyle', 'Doctor_Notes', 'Diagnosis', 'Risk_Level'],
        num_rows: 1500
    })
})

In [4]:
df = pd.DataFrame(dataset['train'])
df.head()

,Patient_ID,Age,Gender,Symptoms,Medical_History,Medications,Lab_Reports,Lifestyle,Doctor_Notes,Diagnosis,Risk_Level
0,P0001,69,Female,"Memory loss, confusion",Stroke history,Ibuprofen,High blood sugar,Healthy diet,"Reports difficulty breathing, requires further...",Cancer Screening Needed,Medium
1,P0002,61,Male,"Shortness of breath, dizziness",Arthritis,Levothyroxine,High blood pressure,Smoker,"Nausea reported, possible gastrointestinal issue",Respiratory Infection,Medium
2,P0003,29,Male,"Shortness of breath, dizziness",Arthritis,Omeprazole,Irregular heartbeat detected,Drinker,"Sudden weight loss unexplained, requires evalu...",Migraine,Low
3,P0004,63,Male,"High fever, body aches",Asthma,Aspirin,High cholesterol,Active,Mild symptoms but family history is concerning,Arthritis,Medium
4,P0005,46,Female,"Joint pain, swelling",No known history,Omeprazole,High blood sugar,Drinker,"Heart palpitations observed, needs ECG",Arthritis,Medium


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Patient_ID       1500 non-null   object
 1   Age              1500 non-null   int64 
 2   Gender           1500 non-null   object
 3   Symptoms         1500 non-null   object
 4   Medical_History  1500 non-null   object
 5   Medications      1338 non-null   object
 6   Lab_Reports      1500 non-null   object
 7   Lifestyle        1500 non-null   object
 8   Doctor_Notes     1500 non-null   object
 9   Diagnosis        1500 non-null   object
 10  Risk_Level       1500 non-null   object
dtypes: int64(1), object(10)
memory usage: 129.0+ KB


In [6]:
classes_heart_disease = ['Heart Disease', 'Hypertension', 'Arrhythmia']
df_heart_disease = df[df['Diagnosis'].isin(classes_heart_disease)].copy()
df_heart_disease.head()

,Patient_ID,Age,Gender,Symptoms,Medical_History,Medications,Lab_Reports,Lifestyle,Doctor_Notes,Diagnosis,Risk_Level
7,P0008,24,Female,"Chest pain, fatigue",Hypertension,Ibuprofen,Irregular heartbeat detected,Sedentary,"Patient forgets daily activities, early dement...",Heart Disease,Low
8,P0009,36,Female,"High fever, body aches",Cancer history,Metformin,Normal MRI,Active,"Joint pain worsening, recommended physical the...",Heart Disease,Medium
9,P0010,35,Female,"Nausea, vomiting",Cancer history,Donepezil,High blood pressure,Healthy diet,"Heart palpitations observed, needs ECG",Heart Disease,High
11,P0012,39,Male,"Rapid heartbeat, sweating",No known history,Levothyroxine,Low white blood cell count,Smoker,"Nausea reported, possible gastrointestinal issue",Hypertension,High
18,P0019,26,Male,"Shortness of breath, dizziness",Stroke history,Aspirin,Brain scan abnormality,Active,Mild symptoms but family history is concerning,Arrhythmia,Low


In [7]:
df_heart_disease.info()

<class 'pandas.core.frame.DataFrame'>
Index: 457 entries, 7 to 1495
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Patient_ID       457 non-null    object
 1   Age              457 non-null    int64 
 2   Gender           457 non-null    object
 3   Symptoms         457 non-null    object
 4   Medical_History  457 non-null    object
 5   Medications      405 non-null    object
 6   Lab_Reports      457 non-null    object
 7   Lifestyle        457 non-null    object
 8   Doctor_Notes     457 non-null    object
 9   Diagnosis        457 non-null    object
 10  Risk_Level       457 non-null    object
dtypes: int64(1), object(10)
memory usage: 42.8+ KB


Binariza o nível de risco para alto risco e baixo risco.

In [8]:
df_heart_disease['Risk_Level_Binary'] = df_heart_disease['Risk_Level'].apply(lambda x: 'High' if x in ['High', 'Medium'] else 'Low')

In [44]:
generator = pipeline('text-generation', 
                     model='EleutherAI/gpt-neo-125M', 
                     max_new_tokens=200, 
                     num_return_sequences=1, 
                     temperature=0.6,
                     top_k=50,
                     do_sample=True, 
                     pad_token_id=50256)

Device set to use cuda:0


In [45]:
def create_prompt(row):
    variacao = random.choice([
        " (You are a 45-year-old office worker.)",
        " (You are a 60-year-old retiree.)",
        " (You are a 30-year-old athlete.)"
    ])

    return f"""You are a patient describing your cardiac symptoms in a first-person narrative to a doctor. Based on the details below, write a single, coherent paragraph (150-200 words) in English. Start with "About a week ago, I started feeling..." and include:
- Vivid description of symptoms ({row['Symptoms']}).
- When symptoms began (e.g., a few days or weeks ago).
- How they impact your daily routine (work, leisure, rest).
- Relevant medical history ({row['Medical_History']}) and lifestyle ({row['Lifestyle']}), woven naturally.
- Reflect the diagnosis ({row['Diagnosis']}): e.g., chest pain for Heart Disease, palpitations for Arrhythmia, fatigue for Hypertension.
- Convey the risk level subtly: for {row['Risk_Level_Binary']}, use an urgent tone for "High Risk" (e.g., "I'm really worried") or a manageable tone for "Low Risk" (e.g., "I cope with rest").
Do not add new symptoms or diagnoses. Keep it realistic, varied, and conversational, like a patient speaking. {variacao}

Case details:
- Symptoms: {row['Symptoms']}
- Medical_History: {row['Medical_History']}
- Lifestyle: {row['Lifestyle']}
- Diagnosis: {row['Diagnosis']}
- Risk_Level: {row['Risk_Level_Binary']}"""

In [51]:
saida = generator("Hello, say me about sky!")

In [53]:
saida[0]['generated_text']

'Hello, say me about sky!\n\nHere, here, and here, are my thoughts on the next topic.\n\n1. How is it that the sky is so beautiful that you can see it right?\n\n2. How is it that the sky is so beautiful that you can see it right?\n\n3. How is it that the sky is so beautiful that you can see it right?\n\n4. How is it that the sky is so beautiful that you can see it right?\n\n5. How is it that the sky is so beautiful that you can see it right?\n\n6. How is it that the sky is so beautiful that you can see it right?\n\n7. How is it that the sky is so beautiful that you can see it right?\n\n8. How is it that the sky is so beautiful that you can see it right?\n\n9. How is it that the sky is so beautiful that you can see it right?\n\n10'

In [46]:
N_variations = 1

In [47]:
generated_texts = []

for idx, row in df_heart_disease[:1].iterrows():
    for _ in range(N_variations):
        prompt = create_prompt(row)

        output = generator(prompt)
        text_en = output[0]['generated_text']

        # Remove o prompt da saída
        if text_en.startswith(prompt):
            text_en = text_en[len(prompt):].strip()
            
        print(text_en)

        generated_texts.append({
            'frase_en': text_en,
            'diagnostico': row['Diagnosis'],
            'risco': row['Risk_Level_Binary']
        })

Dump to the side

#    
"Your heart is hurting so much, the rest of your life is too much"

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V. Khatib

A. V.


In [48]:
df_generated = pd.DataFrame(generated_texts)
df_generated.head()

,frase_en,diagnostico,risco
0,"Dump to the side\n\n# \n""Your heart is hurt...",Heart Disease,Low


In [49]:
df_generated[:10]['frase_en']

0    Dump to the side\n\n#    \n"Your heart is hurt...
Name: frase_en, dtype: object

In [50]:
df_generated[['frase_en', 'diagnostico', 'risco']].to_csv('frases_cardiacas_sinteticas_en.csv', index=False)